# Imorting necisary libraries

In [1]:
# Import
import tensorflow as tf
import numpy as np
import os
import unirep
from data_utils import aa_seq_to_int
import csv
import gc
import pandas as pd
from sklearn.model_selection import train_test_split
from Bio import SeqIO
from Preprocessing import to_binary

#Define
RANDOM_SEED = 42
USE_FULL_1900_DIM_MODEL = True # if True use 1900 dimensional model, else use 64 dimensional one.

# Declaring and initiates Unitep Model

In [2]:

# Set seeds
tf.set_random_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

batch_size = 32
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

/bin/sh: 1: aws: not found
Instructions for updating:
dim is deprecated, use axis instead
(?, 1900)


# Loading data set

In [3]:
file_name = 'test.csv'
unirep_dict = {'id': [], 'property_ogt': [], 'seq':[], 'rep':[]}

if(file_name[-3:]=='csv'):
    with open(file_name, newline='') as csvfile:   
        reader = csv.DictReader(csvfile)
        for i, row in enumerate(reader):
            unirep_dict['id'].append(row['id'])
            unirep_dict['property_ogt'].append(float(row['property_ogt']))
            unirep_dict['seq'].append(row['seq'])
elif file_name[-5:]=='fasta':
    for rec in SeqIO.parse(file_name,'fasta'):
        topt_dict['id'].append(rec.id)
        topt_dict['ogt'].append(float(rec.description.split()[-1]))
        topt_dict['seq'].append(rec.seq)
        topt_dict['seq_bin'].append(to_binary(rec.seq))
    
else:
    print('Data format {} is not suported'.format(file_name.split('.')[1]))
    

# Split train, val and test set and Sort data set by sequense length

In [4]:
# 


X_train, X_Test_Val, y_train, y_Test_Val = train_test_split( unirep_dict['seq'], unirep_dict['property_ogt'], test_size=0.3, random_state=RANDOM_SEED)
X_val, X_test, y_val, y_test = train_test_split(X_Test_Val , y_Test_Val, test_size=0.5, random_state=RANDOM_SEED)

# Sort Training set
sorted_zip_train = sorted(zip(X_train, y_train), key=lambda pair: len(pair[0]))
X_train = [X for X, _ in sorted_zip_train]
y_train = [y for _,y in sorted_zip_train]

# Sort Validation set
sorted_zip_val = sorted(zip(X_val, y_val), key=lambda pair: len(pair[0]))
X_val = [X for X, _ in sorted_zip_val]
y_val = [y for _, y in sorted_zip_val]

# Sort Test set
sorted_zip_test = sorted(zip(X_test, y_test), key=lambda pair: len(pair[0]))
X_test = [X for X, _ in sorted_zip_test]
y_test = [y for _, y in sorted_zip_test]

#  Tuning model

In [5]:
################################# Creating top model #######################################

hid=b._rnn_size     # Not correct way to get hidden variable
dense_nodes_1 = 128 
dense_nodes_2 = 128
prop_size = 20 # should be set to 20 if Evo tuning 

# Variable declaration UniRep mLSTM
labels_prop = tf.placeholder(tf.float32,(None,prop_size), name='Target_prop')
labels_evo = tf.placeholder(tf.float32,(None,prop_size), name='Target_evo')
hidden_states_p = tf.placeholder(tf.float32, (None,None,hid), name='hs')
non_pad_len_p = tf.placeholder(tf.int32, (None,), name='batch_size_hs')
lr = tf.placeholder(tf.float32, shape=[], name='learningrate')
size = tf.placeholder(tf.int32, shape=[], name='Size_of_batch')

# Variable declarations top model
with tf.variable_scope('top_model'):

    beta1= tf.get_variable(shape =[hid],  initializer=tf.initializers.constant(1,dtype=tf.float32), name='Beta_1')
    gamma1= tf.get_variable(shape =[hid],  initializer=tf.initializers.constant(0,dtype=tf.float32), name='gamma_1')

    w1 = tf.get_variable(name='Weights_top1', shape=[hid,dense_nodes_1] )
    b1 = tf.get_variable(shape=[dense_nodes_1], name='Bias_top1')
    
    beta2= tf.get_variable(shape =[dense_nodes_1],  initializer=tf.initializers.constant(1,dtype=tf.float32), name='Beta_2')
    gamma2= tf.get_variable(shape =[dense_nodes_1],  initializer=tf.initializers.constant(0,dtype=tf.float32), name='gamma_2')
    
    w2 = tf.get_variable(shape=[dense_nodes_1,dense_nodes_2], name='Wights_top2')
    b2 = tf.get_variable(shape=[dense_nodes_2], name='Bias_top2')
    
    beta3= tf.get_variable(shape =[dense_nodes_2],  initializer=tf.initializers.constant(1,dtype=tf.float32), name='Beta_3')
    gamma3= tf.get_variable(shape =[dense_nodes_2],  initializer=tf.initializers.constant(0,dtype=tf.float32), name='gamma_3')

    w3 = tf.get_variable(shape=[dense_nodes_2,prop_size], name='Weights_top3')
    b3 = tf.get_variable(shape=[1], name='Bias_top3')

# Get build parameters from UniRep model    
final_hidden, x_placeholder, batch_size_placeholder, seq_length_placeholder, initial_state_placeholder = (
    b.get_rep_ops())

# Generate features as in article
e = lambda j: tf.reduce_mean(tf.gather_nd(hidden_states_p,
                                               tf.stack([tf.ones(non_pad_len_p[j],dtype=tf.int32)*j,
                                                tf.range(non_pad_len_p[j],dtype=tf.int32)], axis=1)), axis=0) #for j in tf.range(size)]
features = tf.map_fn(e,tf.range(size, dtype=tf.int32), dtype=tf.float32)

# Top Model build
mean, variance = tf.nn.moments(final_hidden, axes = [0])
lstm_out = tf.nn.batch_normalization( final_hidden, mean, variance, beta1, gamma1,variance_epsilon = 1e-6,name=None)

dens1 = tf.add(tf.matmul(lstm_out,w1),b1)
mean1, variance1 = tf.nn.moments(dens1, axes = [0])
bn1 = tf.nn.batch_normalization( dens1, mean1, variance1, beta2, gamma2,variance_epsilon = 1e-6,name=None)
akt1 = tf.nn.relu(bn1)
do1 = tf.nn.dropout(akt1, 0.2)

dens2 = tf.add(tf.matmul(do1,w2),b2)
mean2, variance2 = tf.nn.moments(dens2, axes = [0])
bn2 = tf.nn.batch_normalization( dens2, mean2, variance2, beta3, gamma3,variance_epsilon = 1e-6,name=None)
akt2 = tf.nn.relu(bn2)
do2 = tf.nn.dropout(akt2, 0.05)

predictions_prop = tf.add(tf.matmul(do2,w3),b3)
predictions_evo = tf.nn.softmax(tf.add(tf.matmul(do2,w3),b3))

# Loss functions
loss_prop = tf.losses.mean_squared_error(
        labels_prop,
        predictions_prop
        )
loss_evo = tf.losses.mean_squared_error(
        labels_evo,
        predictions_evo
        )

# optimizers and backprop
opt = tf.train.AdamOptimizer(learning_rate=lr)
minimize_prop = opt.minimize(loss=loss_prop)
minimize_evo = opt.minimize(loss=loss_evo)

#Callbacks

SS_res = tf.reduce_sum(tf.square(labels_prop - predictions_prop))
SS_tot = tf.reduce_sum(tf.square(labels_prop - tf.reduce_mean(labels_prop)))
k_coff = 1 - SS_res / (SS_tot + 1e-6)


# Prop tune mLSTM layer (with proper truncated backpropagation through time) 

In [7]:
############################## Training regeme TBT ###################################

# Trunkated backprop parameters
grad_stop = 40 # parameter needs to be larger then the difference within a mini-batch
size_train = (len(X_train))//batch_size # number of minibatches per training epoch
size_val = len(X_val)//batch_size  # number of minibatches per validation epoch
epoch = 10
learningrate = 0.005

# Initiate callback logg arrays
loss_acc = np.zeros(epoch)
loss_acc_val = np.zeros(epoch)
k_coff_acc = np.zeros(epoch)
k_coff_acc_val = np.zeros(epoch)

#Initiate session for Prop tuning
sess = tf.Session()
unirep.initialize_uninitialized(sess)

for k in range(epoch):
    # Training loop
    for i in range(size_train):
        index = np.random.randint(0,size_train-batch_size) # creating random variable for batch creation
        batch = X_train[index:batch_size+index]
        labels = np.array(y_train[index:batch_size+index], dtype=np.float32)
        labels = labels.reshape([labels.shape[0],1])
        int_seq = list(map(aa_seq_to_int, batch))
        
        non_pad_len_whole = np.array([len(int_seq[i]) for i in range(len(int_seq))], dtype=np.int32)
        max_len_batch = np.max(non_pad_len_whole)
        min_len_batch = np.min(non_pad_len_whole)
        non_pad_len_part = np.array([max_len_batch-grad_stop for i in range(len(int_seq))], dtype=np.int32)
        tmp = np.zeros([batch_size, max_len_batch], dtype=np.float32)
        for j, seq in enumerate(int_seq):
            tmp[j,:len(seq)] = seq
        int_seq = tmp

        # Truncated gradient decent through time
        if max_len_batch > grad_stop+1:
            final_state_, fh_ = sess.run([b._final_state, final_hidden], feed_dict={b._batch_size_placeholder: batch_size,
                                                                                b._minibatch_x_placeholder: int_seq[:,:-grad_stop],
                                                                                seq_length_placeholder: non_pad_len_part,
                                                                                b._initial_state_placeholder: b._zero_state})

            
            [_, l,m] = sess.run([minimize_prop, loss_prop, k_coff], feed_dict={b._batch_size_placeholder: batch_size,
                                                                                b._minibatch_x_placeholder: int_seq[:,-grad_stop:],
                                                                                labels_prop: labels,
                                                                                seq_length_placeholder: non_pad_len_whole,
                                                                                b._initial_state_placeholder: (final_state_[0], fh_),
                                                                                lr: learningrate})
        else:
            [_, l,m] = sess.run([minimize_prop, loss, k_coff], feed_dict={b._batch_size_placeholder: batch_size,
                                                                                b._minibatch_x_placeholder: int_seq,
                                                                                labels_prop: labels,
                                                                                seq_length_placeholder: non_pad_len_whole,
                                                                                b._initial_state_placeholder: b._zero_state,
                                                                                lr: learningrate})
        loss_acc[k] += l/size_train
        k_coff_acc[k] += m/size_train
    # Validation loop
    for i in range(size_val):
        
        batch = X_test[i*batch_size:batch_size*i+batch_size]
        labels = np.array(y_test[i*batch_size:batch_size*i+batch_size], dtype=np.float32)
        labels = labels.reshape([labels.shape[0],1])

        int_seq = list(map(aa_seq_to_int, batch))
        non_pad_len_ = np.array([len(int_seq[i]) for i in range(len(int_seq))])
        max_len_batch = np.max(non_pad_len_)
        min_len_batch = np.min(non_pad_len_)
        tmp = np.zeros([batch_size, max_len_batch], dtype=np.float32)
        for j, seq in enumerate(int_seq):
            tmp[j,:len(seq)] = seq
        int_seq = tmp
        
        
        # Validation does not need to be split in two operations 
        [l_val,m_val] = sess.run([loss_prop, k_coff], feed_dict={b._batch_size_placeholder: batch_size,
                                                                            b._minibatch_x_placeholder: int_seq,
                                                                            labels_prop: labels,
                                                                            seq_length_placeholder: non_pad_len_whole,
                                                                            b._initial_state_placeholder: b._zero_state})

        
         
        loss_acc_val[k] += l_val/size_val
        k_coff_acc_val[k] += m_val/size_val
    print('epoch: ', k)
    print('Loss: ', loss_acc[k], 'Coeff_k: ', k_coff_acc[k], 'val_Loss: ', loss_acc_val[k], 'val_Coeff_k: ', k_coff_acc_val[k])
    
    #  Simple learning rate decrease algorithm (Should be upgraded to uppdate on plateau) 
    if k%10 == 0 and k>0:
        learningrate *= 0.8
        print('learning rate: ', learningrate)

epoch:  0
Loss:  1685.2069270436357 Coeff_k:  -3.1540443839096444 val_Loss:  1141.9253005981445 val_Coeff_k:  -2.5947561264038086
epoch:  1
Loss:  869.4872778915776 Coeff_k:  -1.1396325739418587 val_Loss:  827.1458206176758 val_Coeff_k:  -1.505938544869423
epoch:  2
Loss:  747.2097435927975 Coeff_k:  -0.8509958488185231 val_Loss:  748.604850769043 val_Coeff_k:  -1.3671105355024338
epoch:  3
Loss:  708.1513500678831 Coeff_k:  -0.7420313242005139 val_Loss:  680.166618347168 val_Coeff_k:  -1.1381970345973969
epoch:  4
Loss:  670.3283795612616 Coeff_k:  -0.6683197632068542 val_Loss:  654.0018463134766 val_Coeff_k:  -1.0941781848669052
epoch:  5
Loss:  713.2685130049542 Coeff_k:  -0.770207844129423 val_Loss:  725.6993026733398 val_Coeff_k:  -1.2858849614858627
epoch:  6
Loss:  678.9382577291349 Coeff_k:  -0.7105784532500478 val_Loss:  706.0018653869629 val_Coeff_k:  -1.2140212655067444
epoch:  7
Loss:  682.863632574314 Coeff_k:  -0.6632370570810828 val_Loss:  642.0657196044922 val_Coeff_k: 

# Evo tune mLSTM layer (with proper truncated backpropagation through time)

In [23]:
############################## Training regeme TBT ###################################

# Trunkated backprop parameters
grad_stop = 40 # parameter needs to be larger then the difference within a mini-batch
size_train = (len(X_train))//batch_size # number of minibatches per training epoch
size_val = len(X_val)//batch_size  # number of minibatches per validation epoch
epoch = 10
learningrate = 0.005

# Initiate callback logg arrays
loss_acc = np.zeros(epoch)


#Initiate session for Evo tuning
sess = tf.Session()
unirep.initialize_uninitialized(sess)

for k in range(epoch):
    # Training loop
    for i in range(size_train):
        index_batch = np.random.randint(0,size_train-batch_size) # creating random variable for batch creation
        batch = X_train[index_batch:batch_size+index_batch]
        int_seq = list(map(aa_seq_to_int, batch))
        non_pad_len_whole = np.array([len(int_seq[i]) for i in range(batch_size)], dtype=np.int32)      
        max_len_batch = np.max(non_pad_len_whole)
        min_len_batch = np.min(non_pad_len_whole)
        index_len = np.random.randint(20,min_len_batch)
        non_pad_len_part = np.array([index_len-grad_stop for i in range(batch_size)], dtype=np.int32)
        labels = np.zeros((batch_size,20))
        tmp = np.zeros((batch_size,index_len-1))
        for j, val in enumerate(int_seq):
            if val[index_len] <= 20:
                labels[j,val[index_len]-1] = 1
            tmp[j,:] = np.array(val)[:index_len-1]
        int_seq = tmp
        

        # Truncated gradient decent through time
        if index_len-1 > grad_stop:
            final_state_, fh_ = sess.run([b._final_state, final_hidden], feed_dict={b._batch_size_placeholder: batch_size,
                                                                                b._minibatch_x_placeholder: int_seq[:,:-grad_stop],
                                                                                seq_length_placeholder: non_pad_len_part,
                                                                                b._initial_state_placeholder: b._zero_state})

            
            [_, l] = sess.run([minimize_evo, loss_evo], feed_dict={b._batch_size_placeholder: batch_size,
                                                                                b._minibatch_x_placeholder: int_seq[:,-grad_stop:],
                                                                                labels_evo: labels,
                                                                                seq_length_placeholder: non_pad_len_whole,
                                                                                b._initial_state_placeholder: (final_state_[0], fh_),
                                                                                lr: learningrate})
        else:
            [_, l] = sess.run([minimize_evo, loss_evo], feed_dict={b._batch_size_placeholder: batch_size,
                                                                                b._minibatch_x_placeholder: int_seq,
                                                                                labels_evo: labels,
                                                                                seq_length_placeholder: non_pad_len_whole,
                                                                                b._initial_state_placeholder: b._zero_state,
                                                                                lr: learningrate})
        loss_acc[k] += l/size_train

    
    print('epoch: ', k)
    print('Loss: ', loss_acc[k])
    
    #  Simple learning rate decrease algorithm (Should be upgraded to uppdate on plateau) 
    if k%10 == 0 and k>0:
        learningrate *= 0.8
        print('learning rate: ', learningrate)

(32, 85)
(32, 75)
(32, 65)
(32, 87)
(32, 57)
(32, 51)
(32, 41)
(32, 87)
(32, 52)
(32, 113)
(32, 76)
(32, 98)
(32, 61)
(32, 83)
(32, 47)
(32, 53)
(32, 99)
(32, 111)
(32, 76)
(32, 93)
(32, 54)
(32, 54)
(32, 91)
(32, 82)
(32, 67)
(32, 54)
(32, 114)
(32, 41)
epoch:  0
Loss:  0.07312739640474318
(32, 107)
(32, 48)
(32, 107)
(32, 67)
(32, 89)
(32, 101)
(32, 87)
(32, 70)
(32, 118)
(32, 41)
(32, 57)
(32, 60)
(32, 53)
(32, 111)
(32, 92)
(32, 64)
(32, 60)
(32, 68)
(32, 56)
(32, 107)
(32, 54)
(32, 49)
(32, 79)
(32, 57)
(32, 57)
(32, 63)
(32, 46)
(32, 110)
epoch:  1
Loss:  0.07155401904771969
(32, 79)
(32, 57)
(32, 81)
(32, 98)
(32, 103)
(32, 54)
(32, 75)
(32, 67)
(32, 75)
(32, 120)
(32, 46)
(32, 108)
(32, 76)
(32, 46)
(32, 59)
(32, 82)
(32, 81)
(32, 65)
(32, 84)
(32, 48)
(32, 45)
(32, 51)
(32, 83)
(32, 44)
(32, 104)
(32, 117)
epoch:  2
Loss:  0.06987548692197333
(32, 120)
(32, 71)
(32, 119)
(32, 55)
(32, 96)
(32, 128)
(32, 80)
(32, 50)
(32, 46)
(32, 57)
(32, 112)
(32, 97)
(32, 103)
(32, 56)
(32, 

KeyboardInterrupt: 

# Saving new waights

In [7]:
save_only_lstm = True
dir_name = "new_1900_weights"


vs = tf.trainable_variables()
top_var = [i.name for i in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="top")]
for v in vs:
    name = v.name
    if save_only_lstm:
        if name not in top_var:
            value = sess.run(v)
            print(name)
            print(value)
            np.save(os.path.join(dir_name,name.replace('/', '_') + ".npy"), np.array(value))
    else:
        value = sess.run(v)
        print(name)
        print(value)
        np.save(os.path.join(dir_name,name.replace('/', '_') + ".npy"), np.array(value))


embed_matrix:0
[[ 9.4531631e-01 -6.6362572e-01 -2.4727893e-01 -7.9199553e-01
   2.6630306e-01 -6.4838982e-01  4.9781895e-01  3.4055996e-01
   2.1549630e-01 -8.5698748e-01]
 [ 2.4772792e-01  2.9151422e-01 -1.5495698e+00  1.9741470e+00
  -1.4079258e+00  2.2512465e+00 -1.0545133e+00  1.9351069e+00
   4.7975802e-01  2.1520741e+00]
 [-2.3115120e+00 -1.1795886e+00 -1.9046620e+00  2.2483774e-02
  -3.3014011e+00  1.1442558e+00 -9.5683850e-02  5.4421264e-01
   1.6127024e+00  1.1679040e+00]
 [-3.8656466e+00 -3.2561639e-01 -1.9194684e+00  1.8437387e+00
  -1.4309715e+00  1.7897017e+00  3.3095551e-01  1.9604835e-01
  -2.2726191e-02  8.1630892e-01]
 [-2.1821768e+00 -8.5190344e-01 -1.9693266e+00 -7.6698601e-01
  -2.5119345e+00  8.3105826e-01 -8.2373492e-02  2.6420088e+00
   1.0705485e+00  1.4846184e+00]
 [-2.9085495e+00 -1.9665563e+00 -1.3311914e+00  2.5400786e+00
   7.7433699e-01  1.2669100e+00 -1.0392420e+00  2.1624033e+00
   1.6375303e+00 -1.1801726e-01]
 [-1.3858920e+00 -1.9543687e+00 -1.3055754e

# Generate feature reprecentations

In [8]:
# You may run in to allocation problem with large data sets 

# File system setup and initialisation
path = 'features/f_'+str(hid)
name1 = 'TOPT_avg_' + str(hid) + '.npy'
name2 = 'TOPT_final_' + str(hid) + '.npy'
name3 = 'TOPT_cell_' + str(hid) + '.npy'
arr1 = np.zeros([len(unirep_dict['seq']),hid],dtype=np.float32)
arr2 = np.zeros([len(unirep_dict['seq']),hid],dtype=np.float32)
arr3 = np.zeros([len(unirep_dict['seq']),hid],dtype=np.float32)

# sort unirep_dict to spead up algorithm
sorted_zip = sorted(zip(unirep_dict['id'], unirep_dict['property_ogt'], unirep_dict['seq']), key=lambda pair: len(pair[2]))
unirep_dict['id'] = [id_ for id_, _, _ in sorted_zip]
unirep_dict['property_ogt'] = [prop for _ ,prop,_ in sorted_zip]
unirep_dict['seq'] = [seq for _ ,_, seq in sorted_zip]

# Feature generation algorithm
for k in range(0,len(unirep_dict['seq']),batch_size):
    batch = unirep_dict['seq'][k:k+batch_size]
    b_size = len(batch)
    int_seq = list(map(aa_seq_to_int, batch))
    non_pad_len_ = np.array([len(int_seq[i])-1 for i in range(len(int_seq))], dtype=np.int32)
    max_len_batch = np.max(non_pad_len_)
    min_len_batch = np.min(non_pad_len_)
    tmp = np.zeros([batch_size, max_len_batch+1], dtype=np.float32)
    for j, seq in enumerate(int_seq):
        tmp[j,:len(seq)] = seq
    int_seq = tmp

    final_state_, hs = sess.run([b._final_state, b._output], feed_dict={b._batch_size_placeholder: batch_size,
                                                                        b._minibatch_x_placeholder: int_seq,
                                                                        b._initial_state_placeholder: (np.zeros( [batch_size, 1900], dtype=np.float32),
                                                                                                       np.zeros([batch_size,1900], dtype=np.float32))})


    rep1 = sess.run([features], feed_dict={hidden_states_p: hs,
                                            non_pad_len_p: non_pad_len_,
                                            size: non_pad_len_.size})

    arr1[k:k+b_size,:] = rep1[0]
    arr2[k:k+b_size,:] = final_state_[0][0]
    arr3[k:k+b_size,:] = final_state_[1][0]
    print('seq: ', k)

# Saving arrays and orderd id & property
np.save(os.path.join(path,name1),arr1)
np.save(os.path.join(path,name2),arr2)
np.save(os.path.join(path,name3),arr3)
f = open(os.path.join(path,'TOPT_id.txt'),'w')
for i in unirep_dict['id']:
    f.write(i+'\n')
f.close()
f = open(os.path.join(path,'TOPT_property.txt'),'w')
for i in unirep_dict['property_ogt']:
    f.write(str(i)+'\n')
f.close()

seq:  0
seq:  32
seq:  64
seq:  96
seq:  128
seq:  160
seq:  192
seq:  224
seq:  256
seq:  288
seq:  320
seq:  352
seq:  384
seq:  416
seq:  448
seq:  480
seq:  512
seq:  544
seq:  576
seq:  608
seq:  640
seq:  672
seq:  704
seq:  736
seq:  768
seq:  800
seq:  832
seq:  864
seq:  896
seq:  928
seq:  960
seq:  992
seq:  1024
seq:  1056
seq:  1088
seq:  1120
seq:  1152
seq:  1184
seq:  1216
seq:  1248
seq:  1280
seq:  1312
seq:  1344
seq:  1376
seq:  1408
seq:  1440
seq:  1472
seq:  1504
seq:  1536
seq:  1568
seq:  1600
seq:  1632
seq:  1664
seq:  1696
seq:  1728
seq:  1760
seq:  1792
seq:  1824
seq:  1856
seq:  1888
